In [ ]:
# Notebook con el modelo basado en reglas para detección de discapacidad
# https://colab.research.google.com/drive/1ojBRjEEfFik7009h-Gvm2nskoeF7G4uC#scrollTo=E6jGkmTFAT9-

In [ ]:
# Requerimientos del programa

# Realizar un programa que detecte el tipo de discapacidad de una persona, según un cuestionario de 20 preguntas e imprimir un pdf con el resultado.
# El cuestionario debe contener preguntas de opción múltiple, con 5 posibles respuestas cada una.
# El programa debe permitir ingresar el nombre de la persona, el número de cédula y las respuestas a las preguntas.
# El programa debe validar que se ingresen los datos solicitados y que las respuestas a las preguntas sean válidas.
# El programa debe imprimir un pdf con el nombre de la persona, el número de cédula y el resultado del cuestionario.
# El programa debe permitir realizar el cuestionario a varias personas, hasta que el usuario decida salir.
# El programa debe permitir visualizar el pdf generado.

In [ ]:
# Eliminamos las importaciones relacionadas con PDF
import os

# Preguntas divididas por dominios
domains = {
    "Cognición": [
        "¿Tiene dificultades para recordar eventos recientes?",
        "¿Se siente confundido con frecuencia en entornos nuevos o poco familiares?",
        "¿Tiene dificultades para concentrarse en tareas durante períodos prolongados?",
        "¿Tiene problemas para comprender información compleja?",
        "¿Olvida frecuentemente realizar tareas importantes?",
        "¿Tiene problemas para resolver problemas o tomar decisiones?",
        "¿Se siente desorientado en lugares familiares?",
        "¿Tiene dificultades para planificar actividades cotidianas?",
        "¿Tiene problemas para seguir instrucciones múltiples?",
        "¿Se distrae fácilmente incluso en entornos tranquilos?"
    ],
    "Movilidad": [
        "¿Tiene dificultades para caminar largas distancias?",
        "¿Tiene problemas para subir o bajar escaleras sin ayuda?",
        "¿Tiene dificultades para levantarse de una silla sin apoyo?",
        "¿Tiene problemas para agacharse o arrodillarse?",
        "¿Experimenta dolor al moverse o caminar?",
        "¿Tiene problemas para mantenerse de pie durante largos períodos?",
        "¿Usa dispositivos de asistencia para moverse, como bastones o sillas de ruedas?",
        "¿Tiene dificultad para moverse en espacios reducidos?",
        "¿Se siente inseguro al caminar en superficies irregulares?",
        "¿Evita actividades físicas debido a limitaciones de movilidad?"
    ],
    "Cuidado personal": [
        "¿Tiene dificultades para bañarse sin ayuda?",
        "¿Tiene problemas para vestirse de manera independiente?",
        "¿Necesita ayuda para alimentarse?",
        "¿Tiene problemas para realizar tareas de higiene personal?",
        "¿Requiere asistencia para usar el baño?",
        "¿Tiene dificultades para lavarse los dientes o peinarse?",
        "¿Tiene problemas para preparar sus comidas diarias?",
        "¿Necesita ayuda para mantener su espacio personal limpio?",
        "¿Tiene problemas para recordar tomar medicamentos?",
        "¿Evita actividades de cuidado personal debido a dolor o limitaciones físicas?"
    ],
    "Relaciones": [
        "¿Tiene dificultades para mantener amistades?",
        "¿Se siente incomprendido por sus emociones en público?",
        "¿Tiene problemas para iniciar conversaciones?",
        "¿Evita eventos sociales debido a ansiedad?",
        "¿Se siente rechazado en entornos grupales?",
        "¿Tiene problemas para trabajar con otros en equipo?",
        "¿Siente que su discapacidad afecta su relación familiar?",
        "¿Tiene problemas para expresar sus necesidades emocionales?",
        "¿Evita interacciones debido al miedo al juicio social?",
        "¿Tiene dificultades para relacionarse con compañeros de trabajo o estudio?"
    ],
    "Actividades de la vida diaria": [
        "¿Tiene problemas para realizar compras en el mercado?",
        "¿Tiene dificultades para realizar tareas domésticas?",
        "¿Necesita ayuda para administrar sus finanzas personales?",
        "¿Tiene problemas para leer o escribir documentos?",
        "¿Tiene dificultades para usar dispositivos electrónicos?",
        "¿Evita salir solo por miedo a caídas o desorientación?",
        "¿Tiene problemas para manejar situaciones de emergencia?",
        "¿Se siente incapaz de participar en actividades recreativas?",
        "¿Tiene dificultades para mantener un horario regular?",
        "¿Evita actividades al aire libre debido a limitaciones físicas o psicológicas?"
    ],
    "Participación": [
        "¿Tiene problemas para participar en actividades comunitarias?",
        "¿Siente que su discapacidad afecta su trabajo o estudios?",
        "¿Evita lugares concurridos por estrés o ansiedad?",
        "¿Tiene dificultades para participar en eventos familiares?",
        "¿Se siente excluido en entornos grupales?",
        "¿Evita viajar debido a problemas de movilidad?",
        "¿Tiene problemas para contribuir en proyectos comunitarios?",
        "¿Siente que su discapacidad limita sus oportunidades de desarrollo?",
        "¿Se siente incapaz de expresar sus opiniones en público?",
        "¿Evita actividades sociales por miedo al juicio?"
    ]
}

options = [
    "1. Nunca",
    "2. Raramente",
    "3. A veces",
    "4. Frecuentemente",
    "5. Siempre"
]

# Mapeo de dominios a tipos de discapacidad
domain_to_disability = {
    "Cognición": "Discapacidad Cognitiva",
    "Movilidad": "Discapacidad Física",
    "Cuidado personal": "Discapacidad Psicosocial",
    "Relaciones": "Discapacidad Psicosocial",
    "Actividades de la vida diaria": "Discapacidad Psicosocial",
    "Participación": "Discapacidad Psicosocial"
}

# Mapeo de la entrada del usuario a los nuevos puntajes
input_to_score = {
    1: 0,  # Nunca
    2: 1,  # Raramente
    3: 2,  # A veces
    4: 3,  # Frecuentemente
    5: 3   # Siempre
}

# Determinar discapacidad según los puntajes
def determine_disability(scores):
    disabilities = {}
    for domain, score in scores.items():
        average_score = score / len(domains[domain])
        if average_score >= 2:  # Umbral de discapacidad
            disability_type = domain_to_disability[domain]
            disabilities[disability_type] = disabilities.get(disability_type, 0) + 1
    if not disabilities:
        return "Sin discapacidad"
    elif len(disabilities) == 1:
        return list(disabilities.keys())[0]
    else:
        return "Discapacidad Múltiple"

# Programa principal
def main():
    name = input("Nombre del paciente: ")
    id_number = input("Número de identificación: ")
    answers = []
    scores = {domain: 0 for domain in domains}

    for domain, questions in domains.items():
        print(f"\n--- {domain} ---")
        for question in questions:
            print(f"\n{question}")
            print("\n".join(options))  # Mostrar opciones
            while True:
                try:
                    answer = int(input("Seleccione una opción (1-5): "))
                    if 1 <= answer <= 5:
                        score_value = input_to_score[answer]
                        break
                    else:
                        print("Ingrese un número entre 1 y 5.")
                except ValueError:
                    print("Entrada no válida. Por favor, ingrese un número entre 1 y 5.")
            scores[domain] += score_value
            answers.append(score_value)

    # Determinar el tipo de discapacidad
    disability_result = determine_disability(scores)
    print(f"\nResultado de la evaluación: {disability_result}")

    # Imprimir puntuaciones por dominio
    print("\nPuntuaciones por dominio:")
    for domain, score in scores.items():
        print(f"{domain}: {score}")

if __name__ == "__main__":
    main()